## Visualization Code

In [1]:
import numpy as np
import pandas as pd

# Required for large file uploads
chunk_size = 16777216
nbins = 10000

### Used to visualize the min error and max error for second layer. 

Min error is at index 2 in file and Max error at index 3. 

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

def plot_hists(identifier, buckets, batch_size, index_file, percentage): 
    filename = "/home/yash/Desktop/CSE-662/Code/LIS/models/{}/model_params_layer_2_{}_{}.txt".format(identifier, buckets, batch_size)
    
    low = np.inf
    high = -np.inf

    data = []
    
    for index in range((buckets+1) // chunk_size + 1):
        chunk = next(pd.read_csv(filename, skiprows=index * chunk_size+1, chunksize=chunk_size,
                            header = None, sep=' ', dtype=np.float64, index_col = False))
        
        chunk = chunk[chunk[index_file]!=-1]
        chunk = chunk[1:-1]
        low = np.minimum(chunk.iloc[:, index_file].min(), low)
        high = np.maximum(chunk.iloc[:, index_file].max(), high)
        data = chunk.iloc[:, index_file]
    
    bin_edges = np.linspace(low, high, nbins + 1)
    if index_file == 2:
        bin_edges = bin_edges[len(bin_edges)-nbins*percentage//100 :]
    else:        
        bin_edges = bin_edges[:nbins*percentage//100]

    plt.hist(data, bin_edges)
    del bin_edges
    del low
    del high
    plt.savefig("/home/yash/Desktop/CSE-662/Code/LIS/graphs/{}_{}_{}_{}.png".format(identifier, buckets, index_file, percentage))
    plt.clf()

In [3]:
identifier = ["lognormal", "lognormal_1","maps","maps_1","weblogs","weblogs_1"]
buckets = [10000,20000,50000,100000,200000]
percentage = [2,5,10, 20, 40, 50, 100]
batch_size = 32
from matplotlib import pyplot as plt
%matplotlib inline

for i in identifier:
    for b in buckets:
        for p in percentage:
            if i[0] == "w" and b == 200000:
                continue
            plot_hists(i, b, batch_size,2, p)
            plot_hists(i, b, batch_size,3, p)            

<Figure size 432x288 with 0 Axes>

### Visualize the distribution of data for layer-2 (Bucket Size)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import os
import glob

def plot_hists(identifier, buckets, percentage): 
    ur_directory = "/media/yash/Data/CSE662_Data/Buckets/{}/{}".format(identifier, buckets)
    os.chdir(ur_directory)
    data=[]
    for fn in glob.glob('*.csv'):
        with open(fn) as f:
            lines = sum(1 for line in f)
            data.append(lines)

    low = min(data)
    high = max(data)
    bin_edges = np.linspace(low, high, nbins + 1)
  
    data = np.asarray(data, dtype=np.float32)
    
    
    for p in percentage:
        new_bin_edges = bin_edges[:(nbins*p)//100]

        plt.hist(data, new_bin_edges)

        plt.savefig("/home/yash/Desktop/CSE-662/Code/LIS/graphs/{}_{}_{}.png".format(identifier, buckets, p))
        plt.clf()

    del data
    del bin_edges
    del low
    del high

In [ ]:
identifier = ["lognormal", "lognormal_1","maps","maps_1","weblogs","weblogs_1"]
buckets = [10000, 20000, 50000,100000,200000]
percentage = [5, 10, 20, 40, 50, 100]

from matplotlib import pyplot as plt
%matplotlib inline

for i in identifier:
    for b in buckets:
        if i[0] == "w" and b == 200000:
            continue
        plot_hists(i, b, percentage)

### Plot the error for layer-1. This needs to be done using on disk readding method.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import math

def plot_hhh(identifier):
    low = np.inf
    high = -np.inf
    filename = "/media/yash/Data/CSE662_Data/Layer_1_Output/{}/layer_1_op.csv".format(identifier)
            
    for chunk in pd.read_table(filename, header=None, chunksize=16777216//2, sep=' ', dtype=np.float64, index_col = False):
        low = np.minimum(chunk.iloc[:, 3].min(), low)
        high = np.maximum(chunk.iloc[:, 3].max(), high)
    
    bin_edges = np.linspace(low, high, nbins + 1)
    total = np.zeros(nbins, np.uint)

    for chunk in pd.read_table(filename, header=None, chunksize=16777216//2, sep=' ', dtype=np.float64, index_col = False):
        subtotal, e = np.histogram(chunk.iloc[:, 3], bins=bin_edges)
        total += subtotal.astype(np.uint)
    
    plt.bar(bin_edges[:-1], total, width=200)
    plt.savefig("/home/yash/Desktop/CSE-662/Code/LIS/graphs/{}.png".format(identifier))
    plt.clf()
    
plot_hhh("weblogs")
plot_hhh("maps")
plot_hhh("lognormal")